In [132]:
import numpy as np
import pandas as pd

from torch.utils.data import DataLoader, TensorDataset
from torch.nn.functional import mse_loss
from torch.optim import Adam, SGD
from torchmetrics.functional.regression import mean_absolute_error
from lightning import LightningModule, Trainer, LightningDataModule
from lightning.pytorch.loggers import TensorBoardLogger
from lightning.pytorch.callbacks import LearningRateMonitor, TQDMProgressBar, LearningRateFinder, StochasticWeightAveraging
from torch.optim.lr_scheduler import OneCycleLR, ExponentialLR, CosineAnnealingLR, ReduceLROnPlateau, CyclicLR, LinearLR, StepLR, MultiStepLR
from lightning.pytorch.tuner import Tuner
import torch.onnx

import tsai.models.XResNet1dPlus

import json
import seaborn as sns
import seaborn.objects as so
sns.set_theme()

%load_ext tensorboard


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


# Models 

In [133]:
xresnets = {
    "xresnet1d18plus": tsai.models.XResNet1dPlus.xresnet1d18plus,
    "xresnet1d34plus": tsai.models.XResNet1dPlus.xresnet1d34plus,
    "xresnet1d50plus": tsai.models.XResNet1dPlus.xresnet1d50plus,
    "xresnet1d101plus": tsai.models.XResNet1dPlus.xresnet1d101plus,
    "xresnet1d152plus": tsai.models.XResNet1dPlus.xresnet1d152plus,
    "xresnet1d18_deepplus": tsai.models.XResNet1dPlus.xresnet1d18_deepplus,
    "xresnet1d34_deepplus": tsai.models.XResNet1dPlus.xresnet1d34_deepplus,
    "xresnet1d50_deepplus": tsai.models.XResNet1dPlus.xresnet1d50_deepplus,
    "xresnet1d18_deeperplus": tsai.models.XResNet1dPlus.xresnet1d18_deeperplus,
    "xresnet1d34_deeperplus": tsai.models.XResNet1dPlus.xresnet1d34_deeperplus,
    "xresnet1d50_deeperplus": tsai.models.XResNet1dPlus.xresnet1d50_deeperplus,
}

def load_model(arch,seq_len):
    if arch in xresnets:
        model = xresnets[arch](1,2,seq_len) 
    else:
        raise ValueError("Unsupported archticture")
    return model.to("mps")

# Dataset

In [140]:
class CSPDataModule(LightningDataModule):
    def __init__(self, json_file, batch_size = 32, num_workers=0, transform=None, frequency=3000, max_time = .5):
        super().__init__()
        self.json_file = json_file
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.transform = transform
        self.frequency = frequency
        self.max_time = max_time

    def prepare_data(self):
        with open(self.json_file) as f:
          self.data=json.load(f)
          self.indices = {"test":[], "train":[], "exclude":[]}
          for i,d in enumerate(self.data):
            self.indices[d["split"]].append(i)
          max_index = int(self.frequency*self.max_time)
          self.signals = torch.unsqueeze(torch.tensor(np.array([d["signal"] for d in self.data],dtype=np.float32)[:,:max_index]*1000),dim=1)
          self.csps = torch.tensor(np.array([d["csp_label"] for d in self.data],dtype=np.float32)/self.max_time)
          self.labels = np.array([f'#{d["id"]}' for d in self.data])
          self.label_indices = torch.tensor(range(len(self.labels)))
          self.split = np.array([d["split"] for d in self.data])
          self.split_names = {name: idx for idx,name in enumerate(np.unique(self.split))}
          self.split_indices = torch.tensor([self.split_names[name] for name in self.split])
          self.time_range = np.array([0,self.max_time])


    def setup(self, stage="fit"):
        if stage == "fit":
            self.train_data = TensorDataset(self.signals[self.indices["train"]], self.csps[self.indices["train"]])
            self.val_data = TensorDataset(self.signals[self.indices["test"]],self.csps[self.indices["test"]])
                        
        elif stage == "test":
            self.test_data =  TensorDataset(self.signals, self.csps, self.label_indices, self.split_indices)
        
    def train_dataloader(self):
        return DataLoader(self.train_data, batch_size=self.batch_size, num_workers=self.num_workers, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.val_data, batch_size=self.batch_size, num_workers=self.num_workers)
    
    def test_dataloader(self):
        return DataLoader(self.test_data, batch_size=self.batch_size, num_workers=self.num_workers)
    
    def on_before_batch_transfer(self, batch, dataloader_idx):
        if self.transform:
            batch = self.transform(batch)
        return batch
    
    def teardown(self, stage: str):
        # Used to clean-up when the run is finished
        pass

In [141]:
def random_resize(to=512):
    def transform(batch):
        if len(batch)==2:
            x,y = batch
            p = []
        else:
            x,y,*p = batch
        length = x.shape[2]
        min_time, max_time = y[:,0].min(), y[:,1].max()
        start_index = torch.randint(low=0, high=int(.8*min_time*length), size=(1,)).item()
        end_index = torch.randint(low=int(1.2*max_time*length), high=length, size=(1,)).item()
        start_time, end_time = start_index/length, end_index/length
        x_transform = torch.nn.functional.interpolate(x[:,:,start_index:end_index], size=(to,),mode="linear",align_corners=True) 
        y_transform = (y-start_time)/(end_time-start_time)
        return x_transform, y_transform, *p
    return transform

In [153]:
def dataset_to_df(dataset):
    data=dataset[:]
    if len(data)==2:
        x,y=data
        idx,split = None, None, None
    elif len(data)==4:
        x,y,idx,split=data
        idx = idx.detach().numpy().tolist()
        split = split.detach().numpy().tolist()
    signal = x.detach().numpy()[:,0].tolist()
    time = np.linspace(0,1,x.shape[2])
    csp = y.detach().numpy().tolist()
    df = pd.DataFrame({"signal": signal, "csp": csp, "idx": idx, "split": split})
    df['time'] = df.apply(lambda x: time, axis=1)
    return df

def plot(df,markers={}):
    df=df.explode(["signal","time"]).reset_index()
    for name in markers.keys():
        df=pd.concat([df,pd.DataFrame(df[name].tolist(),columns=[name+"_on",name+"_off"])],axis=1)
    g = (
        so.Plot(data=df, x="time", y="signal")
        .add(so.Line())
        .facet(row="index")
        .limit(y=(-1, 1))
    )
    for name,color in markers.items():
        g = g.add(so.Range(color=color), x=name+"_on", ymin=-1, ymax=1).add(so.Range(color=color), x=name+"_off", ymin=-1, ymax=1)
    return g

# Learner

In [202]:
class CSPLearner(LightningModule):
    def __init__(self, model, **kwargs):
        super().__init__()
        self.save_hyperparameters(ignore=['model'])
        self.model = model

    def forward(self, x):
        out = self.model(x)
        return out

    def evaluate(self, batch, stage=None):
        x, y, *_ = batch
        z = self(x)
        loss = mse_loss(y, z)
        mae = mean_absolute_error(y,z)

        if stage is not None:
            self.log(f"{stage}_loss", loss, prog_bar=True)
            self.log(f"{stage}_mae", mae, prog_bar=True)

        return loss

    def training_step(self, batch, batch_idx):
        return self.evaluate(batch, stage="train")

    def validation_step(self, batch, batch_idx):
        return self.evaluate(batch, stage="val")

    def test_step(self, batch, batch_idx):
        x, y, idx, split = batch
        z = self(x)
        self.test_pred["csp"] += y.detach().cpu().numpy().tolist()
        self.test_pred["csp_pred"] += z.detach().cpu().numpy().tolist()
        self.test_pred["idx"] += idx.detach().cpu().numpy().tolist()
        self.test_pred["split"] += split.detach().cpu().numpy().tolist()
        return (mse_loss(y,z), mean_absolute_error(y,z))
    
    def on_train_start(self):
        self.logger.log_hyperparams(self.hparams, {"val_loss": 1, "train_loss": 1})
    
    def on_test_start(self):
        self.test_pred = {"csp": [], "csp_pred": [], "idx": [], "split": []}
        

    def on_test_end(self) -> None:
        df=pd.DataFrame(self.test_pred)
        #df["label"] = cspdata.labels[df["idx"]]
        #df["split"] = np.array(list(cspdata.split_names.keys()))[df["split"]]
        df=df.explode(["csp","csp_pred"])
        df["type"] = ["on","off"]*(len(df)//2)
        df["error"] = np.abs(df["csp"]-df["csp_pred"])
        self.df = df 
        print(df.groupby("split").agg({"error": ["mean","std"]}))
        

    def configure_optimizers(self):
        #optimizer = SGD(self.parameters(), lr=self.hparams.lr, momentum=self.hparams.momentum, weight_decay=self.hparams.weight_decay)
        optimizer = Adam(
            self.parameters(),
            lr=self.hparams.lr,
        )
        scheduler = OneCycleLR(optimizer, 
                               max_lr=self.hparams.max_lr, 
                               epochs=self.trainer.max_epochs, 
                               steps_per_epoch=1,
                               three_phase=self.hparams.three_phase,
                               )
        #Adam(
        #     self.parameters(),
        #     lr=self.hparams.lr,
        #)
        #scheduler = ExponentialLR(optimizer=optimizer, gamma=self.hparams.gamma, last_epoch=-1) 
        #scheduler = CosineAnnealingLR(optimizer=optimizer, T_max=200)
        #scheduler = OneCycleLR(optimizer, 
        #                       max_lr=self.hparams.lr, 
        #                       epochs=self.trainer.max_epochs, 
        #                       steps_per_epoch=1,
        #                       three_phase=self.hparams.three_phase)
        return {"optimizer": optimizer, 'lr_scheduler': scheduler}


# Training

In [214]:
window_size = 512
batch_size  = 400
lr = 1e-4
epochs = 2000
arch = "xresnet1d18plus"
accumulate_grad_batches = 1
max_lr = 1e-3

model = load_model("xresnet1d18plus", window_size)

cspdata = CSPDataModule(
        "all_data.json", 
        batch_size=batch_size, 
        num_workers=0, 
        transform=random_resize(to=window_size)
        )

learner = CSPLearner(
        model, 
        lr=lr, 
        batch_size=batch_size, 
        window_size=window_size, 
        epochs=epochs, 
        accumulate_grad_batches=accumulate_grad_batches, 
        max_lr = max_lr,
        three_phase = False,
        )

trainer = Trainer(
        max_epochs=epochs,
        accelerator="auto",
        devices=1,
        logger=TensorBoardLogger("lightning_logs", name=arch, default_hp_metric=False),
        callbacks=[
            LearningRateMonitor(logging_interval="epoch"), 
            LearningRateFinder(),
            TQDMProgressBar(refresh_rate=10), 
            #StochasticWeightAveraging(swa_lrs=1e-6),
            ],
        log_every_n_steps=1,
        #profiler="simple",
        accumulate_grad_batches=accumulate_grad_batches,
    )

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [215]:
trainer.fit(learner, cspdata)

/Users/dschroeder/miniconda3/envs/ml/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
/Users/dschroeder/miniconda3/envs/ml/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=100` reached.
Learning rate set to 0.0001445439770745928
Restoring states from the checkpoint path at /Users/dschroeder/Documents/csp/.lr_find_e4328cad-a575-47ae-8868-a690e7dbc402.ckpt
Restored all states from the checkpoint at /Users/dschroeder/Documents/csp/.lr_find_e4328cad-a575-47ae-8868-a690e7dbc402.ckpt

  | Name  | Type          | Params
----------------------------------------
0 | model | XResNet1dPlus | 3.9 M 
----------------------------------------
3.9 M     Trainable params
0         Non-trainable params
3.9 M     Total params
15.416    Total estimated model params size (MB)
Restored all states from the checkpoint at /Users/dschroeder/Documents/csp/.lr_find_e4328cad-a575-47ae-8868-a690e7dbc402.ckpt


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

# Evaluation

In [212]:
trainer.test(learner, cspdata)

/Users/dschroeder/miniconda3/envs/ml/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

          error          
           mean       std
split                    
0      0.050731  0.050829
1      0.023262  0.024748
2      0.016472  0.012481


[{}]

In [213]:
np.array(list(cspdata.split_names.keys()))

array(['exclude', 'test', 'train'], dtype='<U7')

error          
             mean       std
split                      
exclude  0.162733  0.132108
test     0.213524  0.149176
train    0.210947  0.152950

In [78]:
def train(
        batch_size=128, 
        epochs=500, 
        lr=1e-4, 
        arch="xresnet1d18plus", 
        window_size=256, 
        accumulate_grad_batches=1, 
        swa_lr=None, 
        max_lr = 1e-2
        ):

    cspdata = CSPDataModule(
        "all_data.json", 
        batch_size=batch_size, 
        num_workers=0, 
        transform=random_resize(to=window_size)
        )
    
    model = load_model(arch, window_size)

    learner = CSPLearner(
        model, 
        lr=lr, 
        batch_size=batch_size, 
        window_size=window_size, 
        epochs=epochs, 
        accumulate_grad_batches=accumulate_grad_batches, 
        max_lr = max_lr,
        momentum = 0,
        weight_decay = 0,
        three_phase = False,
        gamma = 
        )
    
    trainer = Trainer(
        max_epochs=epochs,
        accelerator="auto",
        devices=1,
        logger=TensorBoardLogger("lightning_logs", name=arch, default_hp_metric=False),
        callbacks=[
            LearningRateMonitor(logging_interval="epoch"), 
            TQDMProgressBar(refresh_rate=10), 
            StochasticWeightAveraging(swa_lrs=swa_lr) if swa_lr else None,
            ],
        log_every_n_steps=1,
        #profiler="simple",
        accumulate_grad_batches=accumulate_grad_batches,
    )
    trainer.fit(learner, cspdata)